In [48]:
#This entire notebook shouldn't take more than 2 hours to run
import requests
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import numpy as npy
import re
from datetime import datetime, timedelta, date
import random

In [26]:
#updated scraped fight data (after running fight_hist_updated function from UFC_data_scraping file)
fight_hist=pd.read_csv('fight_hist.csv',sep=',')
#all stats fight history file which is one update behind fight_hist
ufcfightscrap=pd.read_csv('ufc_fights_crap.csv',sep=',')
#updated scraped fighter data (after running fight_hist_updated function from UFC_data_scraping file)
ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')
#most recent fight in fight_hist versus most recent fight in ufcfightscrap

In [55]:
#this cell contains all functions defined for building columns in ufcfightscrap
#converts from '%B %d, %Y' (i.e. August 22, 2020) to date (i.e. 2020-08-22)
def convert_to_datetime(day1):
    return datetime.strptime(day1, '%B %d, %Y').date()
convert_to_datetime_vect= npy.vectorize(convert_to_datetime)

def age(birthDate,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    if birthDate=='--':
        aa='unknown'
    elif type(birthDate)==str and not type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif type(birthDate)==str and type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif not type(birthDate)==str and type(day)==str:
        bd=birthDate
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    else:
        bd=birthDate
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    return aa

age_vect= npy.vectorize(age)

def fighter_age(fighter,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            dob=datetime.strptime(ufcfighterscrap['dob'][i], '%b %d, %Y').strftime('%B %d, %Y')
            a=age(dob,day,form1,form2)
            break
    return a

fighter_age_vect= npy.vectorize(fighter_age)

def wins_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
    return summ

def losses_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
    return summ

#functions for height and reach
def fighter_height(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['height'][i]
            break
    if a=='--':
        b='unknown'
    elif a[4]=='"':
        b=int(a[0])*30.48+int(a[3])*2.54
    else:
        b=int(a[0])*30.48+int(a[3]+a[4])*2.54
    return b

def fighter_reach(fighter):
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['reach'][i]
            break
    if a=='--':
        b='unknown'
    else:
        b=int(a[0]+a[1])*2.54
    return b



wins_before_vect= npy.vectorize(wins_before)
losses_before_vect= npy.vectorize(losses_before)
fighter_height_vect= npy.vectorize(fighter_height)
fighter_reach_vect= npy.vectorize(fighter_reach, otypes=[object])


#day1 should be input in the form '%B %d, %Y' i.e. 'August 20, 1962'
#conversions can be made via day=datetime.strptime(ufcfightsML_known_df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
def time_diff(day1,day2=date.today()):
    if day2==date.today():
        answer=(day2-datetime.strptime(day, '%B %d, %Y')).days
    else:
        answer=(datetime.strptime(day2, '%B %d, %Y')-datetime.strptime(day1, '%B %d, %Y')).days
    return answer

#we now vectorize this to use in pandas/numpy
time_diff_vect= npy.vectorize(time_diff)

#can make a single function to do all of these... actually maybe the count function would even work as is

def L5Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

#vectorize all of these functions
L5Y_wins_vect= npy.vectorize(L5Y_wins)
L5Y_losses_vect= npy.vectorize(L5Y_losses)
L2Y_wins_vect= npy.vectorize(L2Y_wins)
L2Y_losses_vect= npy.vectorize(L2Y_losses)
ko_wins_vect= npy.vectorize(ko_wins)
ko_losses_vect= npy.vectorize(ko_losses)
L5Y_ko_wins_vect= npy.vectorize(L5Y_ko_wins)
L5Y_ko_losses_vect= npy.vectorize(L5Y_ko_losses)
L2Y_ko_wins_vect= npy.vectorize(L2Y_ko_wins)
L2Y_ko_losses_vect= npy.vectorize(L2Y_ko_losses)
sub_wins_vect= npy.vectorize(sub_wins)
sub_losses_vect= npy.vectorize(sub_losses)
L5Y_sub_wins_vect= npy.vectorize(L5Y_sub_wins)
L5Y_sub_losses_vect= npy.vectorize(L5Y_sub_losses)
L2Y_sub_wins_vect= npy.vectorize(L2Y_sub_wins)
L2Y_sub_losses_vect= npy.vectorize(L2Y_sub_losses)

#forgot to change this from ufcfightscrap to test_new_rows... caused a big problem in opponent columns
#careful about this in the future (and check for similar mistakes)
#for columns like fighter_rec which contains the information for the opponent as well, we use the following
def opponent_column(stat):
    col=dict()
    for i in range(len(test_new_rows['fighter'])):
        if i%2==0:
            col[i]=test_new_rows[stat][i+1]
        else:
            col[i]=test_new_rows[stat][i-1]
    statdict={'stat':col}
    return pd.DataFrame (statdict, columns = ['stat'])

#enter date unabbreviated 'July 4, 2019'
#here the average gives avg per fight. Later in avg_count we change to average per time spent in octagon
def count(stat, guy,inf_abs, total_L5Y_L2Y_avg, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if total_L5Y_L2Y_avg=='total' or total_L5Y_L2Y_avg=='avg':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0]
    elif total_L5Y_L2Y_avg=='L2Y':
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<730]
    else:
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<1825]
    if inf_abs=='inf':
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    else:
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy]
    good_indices=[i for i in good_indices_1 if i in good_indices_2]
    if total_L5Y_L2Y_avg!='avg':
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
    else:
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
        day1=convert_date_to_abbrev(day1)
        number_fights=wins_before(guy,day1)+losses_before(guy,day1)
        summ=summ/float(number_fights)
    return summ
        
#note a better average is per time not per fight. We will do each stat as an average per one minutes

#enter time in the form 'August 24, 2018'
def time_in_octagon(guy,day1=date.today().strftime('%B %d, %Y')):
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['fighter'][i]==guy] 
    for i in good_indices:
        if ufcfightscrap['time'][i][2]==':':
            summ=int(ufcfightscrap['time'][i][0:2])+int(ufcfightscrap['time'][i][3:])/60.0
        else:
            summ+=5*(ufcfightscrap['round'][i]-1)+int(ufcfightscrap['time'][i][0])+int(ufcfightscrap['time'][i][2:])/60.0
    return summ

#enter date unabbreviated 'July 4, 2019'
#gives takedowns per minute
def avg_count(stat, guy,inf_abs, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if inf_abs=='inf':
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    else:
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    for i in good_indices:
        summ+=ufcfightscrap[stat][i]
    t= time_in_octagon(guy,day1)
    if t==0:
        summ=0
    else:
        summ=summ/t
    return summ

#vectorize these functions
count_vect= npy.vectorize(count)
avg_count_vect= npy.vectorize(avg_count)

 

In [28]:
update_time=time_diff(ufcfightscrap['date'][0],fight_hist['date'][0])
print(update_time)

14


In [29]:
#this gives the new rows in fight_hist which do not appear in ufcfightscrapd
new_rows=fight_hist.loc[time_diff_vect(fight_hist['date'],fight_hist['date'][0])<update_time]
#convert to numpy array
numpy_new_rows=new_rows.values
#gives new_rows as a dataframe instead of a clone
test_new_rows=pd.DataFrame(data=numpy_new_rows,    # 1st column as index
                columns=new_rows.columns)  # 1st row as the column names

In [30]:
#adding a datetime column for test_new_rows
#test_new_rows['datetime']=convert_to_datetime_vect(test_new_rows['date'])
#setting up columns for rec,wins,losses,ties (can be done in one line each as we do later with other stats)
#run time for each is 20 min per pair
test_new_rows['fighter_wins']=wins_before_vect(test_new_rows['fighter'],test_new_rows['date'])
#column for fighter losses in test_new_rows
test_new_rows['fighter_losses']=losses_before_vect(test_new_rows['fighter'],test_new_rows['date'])
#column for fighter age in test_new_rows
test_new_rows['fighter_age']=fighter_age_vect(test_new_rows['fighter'],test_new_rows['date'])
#column for fighter height in test_new_rows
test_new_rows['fighter_height']=fighter_height_vect(test_new_rows['fighter'])
#column for fighter reach in test_new_rows
test_new_rows['fighter_reach']=fighter_reach_vect(test_new_rows['fighter'])

In [33]:
test_new_rows

,date,division,event_url,fight_url,fighter,fighter_url,method,opponent,opponent_url,result,...,distance_strikes_attempts,clinch_strikes_landed,clinch_strikes_attempts,ground_strikes_landed,ground_strikes_attempts,fighter_wins,fighter_losses,fighter_age,fighter_height,fighter_reach
0,"August 29, 2020",Light Heavyweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/1ff4586752ee...,Aleksandar Rakic,http://ufcstats.com/fighter-details/333b9e5c72...,U-DEC,Anthony Smith,http://ufcstats.com/fighter-details/d4c9dcd330...,W,...,34,2,2,17,21,4,1,28,193.04,198.12
1,"August 29, 2020",Light Heavyweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/1ff4586752ee...,Anthony Smith,http://ufcstats.com/fighter-details/d4c9dcd330...,U-DEC,Aleksandar Rakic,http://ufcstats.com/fighter-details/333b9e5c72...,L,...,14,1,2,2,5,8,5,32,193.04,193.04
2,"August 29, 2020",Welterweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/864cddd6e165...,Neil Magny,http://ufcstats.com/fighter-details/84b3e7d38f...,U-DEC,Robbie Lawler,http://ufcstats.com/fighter-details/f2925e6db4...,W,...,63,16,18,7,7,16,6,33,190.50,203.2
3,"August 29, 2020",Welterweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/864cddd6e165...,Robbie Lawler,http://ufcstats.com/fighter-details/f2925e6db4...,U-DEC,Neil Magny,http://ufcstats.com/fighter-details/84b3e7d38f...,L,...,33,6,6,5,6,13,8,38,180.34,187.96
4,"August 29, 2020",Women's Flyweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/0a9f22de8600...,Alexa Grasso,http://ufcstats.com/fighter-details/e8b731feff...,U-DEC,Ji Yeon Kim,http://ufcstats.com/fighter-details/a20f234df7...,W,...,174,3,3,4,5,3,3,27,165.10,167.64
5,"August 29, 2020",Women's Flyweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/0a9f22de8600...,Ji Yeon Kim,http://ufcstats.com/fighter-details/a20f234df7...,U-DEC,Alexa Grasso,http://ufcstats.com/fighter-details/e8b731feff...,L,...,184,3,4,0,0,3,2,30,170.18,182.88
6,"August 29, 2020",Featherweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/3a8e4b830750...,Ricardo Lamas,http://ufcstats.com/fighter-details/3974fa35c9...,U-DEC,Bill Algeo,http://ufcstats.com/fighter-details/9abc648e76...,W,...,108,14,15,28,33,10,6,38,172.72,180.34
7,"August 29, 2020",Featherweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/3a8e4b830750...,Bill Algeo,http://ufcstats.com/fighter-details/9abc648e76...,U-DEC,Ricardo Lamas,http://ufcstats.com/fighter-details/3974fa35c9...,L,...,145,16,21,2,2,0,0,31,182.88,185.42
8,"August 29, 2020",Middleweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/eb9097be07a3...,Impa Kasanganay,http://ufcstats.com/fighter-details/257b0c8d6f...,U-DEC,Maki Pitolo,http://ufcstats.com/fighter-details/c549573c0d...,W,...,190,3,8,0,0,0,0,26,180.34,190.5
9,"August 29, 2020",Middleweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/eb9097be07a3...,Maki Pitolo,http://ufcstats.com/fighter-details/c549573c0d...,U-DEC,Impa Kasanganay,http://ufcstats.com/fighter-details/257b0c8d6f...,L,...,172,0,1,0,0,1,2,29,177.80,190.5


In [34]:
#making columns for fighter_L5Y_wins losses etc
test_new_rows['fighter_L5Y_wins']=L5Y_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_losses']=L5Y_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_wins']=L2Y_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_losses']=L2Y_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_ko_wins']=ko_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_ko_losses']=ko_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_ko_wins']=L5Y_ko_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_ko_losses']=L5Y_ko_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_ko_wins']=L2Y_ko_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_ko_losses']=L2Y_ko_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_sub_wins']=sub_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_sub_losses']=sub_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_sub_wins']=L5Y_sub_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L5Y_sub_losses']=L5Y_sub_losses_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_sub_wins']=L2Y_sub_wins_vect(test_new_rows['fighter'],test_new_rows['date'])
test_new_rows['fighter_L2Y_sub_losses']=L2Y_sub_losses_vect(test_new_rows['fighter'],test_new_rows['date'])

In [35]:
#forming columns for fighter_inf_knockdowns_avg etc...
test_new_rows['fighter_inf_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_pass_avg']=avg_count_vect('pass',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_reversals_avg']=avg_count_vect('reversals',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['fighter'],'inf',test_new_rows['date'])
test_new_rows['fighter_inf_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['fighter'],'inf',test_new_rows['date'])


In [36]:
#forming columns for fighter_abs_knockdowns_avg etc...
test_new_rows['fighter_abs_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_pass_avg']=avg_count_vect('pass',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_reversals_avg']=avg_count_vect('reversals',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['fighter'],'abs',test_new_rows['date'])
test_new_rows['fighter_abs_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['fighter'],'abs',test_new_rows['date'])

In [37]:
#opponent columns for wins etc
test_new_rows['opponent_wins']=opponent_column('fighter_wins')
test_new_rows['opponent_losses']=opponent_column('fighter_losses')
test_new_rows['opponent_age']=opponent_column('fighter_age')
test_new_rows['opponent_height']=opponent_column('fighter_height')
test_new_rows['opponent_reach']=opponent_column('fighter_reach')
#making columns for opponent_L5Y_wins losses etc
test_new_rows['opponent_L5Y_wins']=opponent_column('fighter_L5Y_wins')
test_new_rows['opponent_L5Y_losses']=opponent_column('fighter_L5Y_losses')
test_new_rows['opponent_L2Y_wins']=opponent_column('fighter_L2Y_wins')
test_new_rows['opponent_L2Y_losses']=opponent_column('fighter_L2Y_losses')
test_new_rows['opponent_ko_wins']=opponent_column('fighter_ko_wins')
test_new_rows['opponent_ko_losses']=opponent_column('fighter_ko_losses')
test_new_rows['opponent_L5Y_ko_wins']=opponent_column('fighter_L5Y_ko_wins')
test_new_rows['opponent_L5Y_ko_losses']=opponent_column('fighter_L5Y_ko_losses')
test_new_rows['opponent_L2Y_ko_wins']=opponent_column('fighter_L2Y_ko_wins')
test_new_rows['opponent_L2Y_ko_losses']=opponent_column('fighter_L2Y_ko_losses')
test_new_rows['opponent_sub_wins']=opponent_column('fighter_sub_wins')
test_new_rows['opponent_sub_losses']=opponent_column('fighter_sub_losses')
test_new_rows['opponent_L5Y_sub_wins']=opponent_column('fighter_L5Y_sub_wins')
test_new_rows['opponent_L5Y_sub_losses']=opponent_column('fighter_L5Y_sub_losses')
test_new_rows['opponent_L2Y_sub_wins']=opponent_column('fighter_L2Y_sub_wins')
test_new_rows['opponent_L2Y_sub_losses']=opponent_column('fighter_L2Y_sub_losses')

In [38]:
#forming columns for opponent_inf_knockdowns_avg etc...
test_new_rows['opponent_inf_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_pass_avg']=avg_count_vect('pass',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_reversals_avg']=avg_count_vect('reversals',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['opponent'],'inf',test_new_rows['date'])
test_new_rows['opponent_inf_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['opponent'],'inf',test_new_rows['date'])

In [39]:
#forming columns for opponent_abs_knockdowns_avg etc...
test_new_rows['opponent_abs_knockdowns_avg']=avg_count_vect('knockdowns',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_pass_avg']=avg_count_vect('pass',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_reversals_avg']=avg_count_vect('reversals',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_sub_attempts_avg']=avg_count_vect('sub_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_takedowns_landed_avg']=avg_count_vect('takedowns_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',test_new_rows['opponent'],'abs',test_new_rows['date'])
test_new_rows['opponent_abs_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',test_new_rows['opponent'],'abs',test_new_rows['date'])

In [40]:
test_new_rows

,date,division,event_url,fight_url,fighter,fighter_url,method,opponent,opponent_url,result,...,opponent_abs_body_strikes_landed_avg,opponent_abs_body_strikes_attempts_avg,opponent_abs_leg_strikes_landed_avg,opponent_abs_leg_strikes_attempts_avg,opponent_abs_distance_strikes_landed_avg,opponent_abs_distance_strikes_attempts_avg,opponent_abs_clinch_strikes_landed_avg,opponent_abs_clinch_strikes_attempts_avg,opponent_abs_ground_strikes_landed_avg,opponent_abs_ground_strikes_attempts_avg
0,"August 29, 2020",Light Heavyweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/1ff4586752ee...,Aleksandar Rakic,http://ufcstats.com/fighter-details/333b9e5c72...,U-DEC,Anthony Smith,http://ufcstats.com/fighter-details/d4c9dcd330...,W,...,0.815483,1.128629,1.161248,1.330869,2.759596,5.754050,0.541481,0.815483,1.474394,1.813635
1,"August 29, 2020",Light Heavyweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/1ff4586752ee...,Anthony Smith,http://ufcstats.com/fighter-details/d4c9dcd330...,U-DEC,Aleksandar Rakic,http://ufcstats.com/fighter-details/333b9e5c72...,L,...,0.280749,0.461230,0.721925,0.802139,1.724599,4.351604,0.501337,0.641711,0.180481,0.200535
2,"August 29, 2020",Welterweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/864cddd6e165...,Neil Magny,http://ufcstats.com/fighter-details/84b3e7d38f...,U-DEC,Robbie Lawler,http://ufcstats.com/fighter-details/f2925e6db4...,W,...,0.906944,1.342702,0.885687,1.101795,3.532121,9.788616,0.924658,1.410014,0.109825,0.162966
3,"August 29, 2020",Welterweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/864cddd6e165...,Robbie Lawler,http://ufcstats.com/fighter-details/f2925e6db4...,U-DEC,Neil Magny,http://ufcstats.com/fighter-details/84b3e7d38f...,L,...,0.343656,0.577795,0.536254,0.634441,1.469033,3.999245,0.268127,0.475831,0.434290,0.623112
4,"August 29, 2020",Women's Flyweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/0a9f22de8600...,Alexa Grasso,http://ufcstats.com/fighter-details/e8b731feff...,U-DEC,Ji Yeon Kim,http://ufcstats.com/fighter-details/a20f234df7...,W,...,1.257442,2.071922,1.000238,1.314599,4.129555,10.473922,0.514408,0.914503,0.100024,0.185759
5,"August 29, 2020",Women's Flyweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/0a9f22de8600...,Ji Yeon Kim,http://ufcstats.com/fighter-details/a20f234df7...,U-DEC,Alexa Grasso,http://ufcstats.com/fighter-details/e8b731feff...,L,...,1.029160,1.569468,0.321612,0.501715,2.765866,8.747856,0.540309,0.720412,0.308748,0.398799
6,"August 29, 2020",Featherweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/3a8e4b830750...,Ricardo Lamas,http://ufcstats.com/fighter-details/3974fa35c9...,U-DEC,Bill Algeo,http://ufcstats.com/fighter-details/9abc648e76...,W,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"August 29, 2020",Featherweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/3a8e4b830750...,Bill Algeo,http://ufcstats.com/fighter-details/9abc648e76...,U-DEC,Ricardo Lamas,http://ufcstats.com/fighter-details/3974fa35c9...,L,...,0.612054,0.924326,0.393463,0.530863,2.092224,5.820756,0.493390,0.711981,0.293536,0.355990
8,"August 29, 2020",Middleweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/eb9097be07a3...,Impa Kasanganay,http://ufcstats.com/fighter-details/257b0c8d6f...,U-DEC,Maki Pitolo,http://ufcstats.com/fighter-details/c549573c0d...,W,...,0.201207,0.321932,0.160966,0.160966,2.454728,5.794769,0.160966,0.160966,0.281690,0.442656
9,"August 29, 2020",Middleweight,http://ufcstats.com/event-details/e29cf523ebd1...,http://ufcstats.com/fight-details/eb9097be07a3...,Maki Pitolo,http://ufcstats.com/fighter-details/c549573c0d...,U-DEC,Impa Kasanganay,http://ufcstats.

In [41]:
#joining test_new_rows to ufcfightscrap to get the most updated version
frames = [test_new_rows, ufcfightscrap]
updated_ufcfightscrap = pd.concat(frames,ignore_index=True)

In [53]:
#saving the updated ufcfightscrap file
updated_ufcfightscrap.to_csv('ufc_fights_crap.csv', index = False)

In [44]:
#here is the list of all stats available (besides stance), does not include names or result
computed_statistics=[u'fighter_wins', u'fighter_losses', u'fighter_age',u'fighter_height',
                    u'fighter_reach', u'fighter_L5Y_wins',u'fighter_L5Y_losses', u'fighter_L2Y_wins', u'fighter_L2Y_losses',
                    u'fighter_ko_wins',u'fighter_ko_losses',u'fighter_L5Y_ko_wins',u'fighter_L5Y_ko_losses',u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',u'fighter_sub_wins',u'fighter_sub_losses',u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', u'fighter_L2Y_sub_wins', u'fighter_L2Y_sub_losses',
                    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    u'opponent_wins', u'opponent_losses', u'opponent_age',  u'opponent_height',
                    u'opponent_reach',   u'opponent_L5Y_wins', u'opponent_L5Y_losses', u'opponent_L2Y_wins', u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', u'opponent_ko_losses', u'opponent_L5Y_ko_wins', u'opponent_L5Y_ko_losses', u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', u'opponent_sub_wins', u'opponent_sub_losses',u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', u'opponent_L2Y_sub_wins', u'opponent_L2Y_sub_losses',
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']



In [45]:
#list containing all columns of any interest
relevant_list=['date','division','fighter','opponent','result','method']
relevant_list.extend(computed_statistics)

In [46]:
#creates a clean file with only columns which are relevant to predicting
updated_ufc_fights=updated_ufcfightscrap[relevant_list]

In [49]:
#lets randomly remove one of every two copied fights
random_indices=[]
for i in range(0,len(updated_ufc_fights['fighter_wins']),2):
    random_indices.append(random.choice([i,i+1]))
    
updated_ufc_fights=updated_ufc_fights.drop(random_indices)

In [54]:
#we worked hard to build this, lets save it
updated_ufc_fights.to_csv('ufc_fights.csv', index = False)

In [ ]:
#add code here to automatically send these to github

In [ ]:
#So the only files we really need are fight_hist,fighter_stats,ufc_fights,ufc_fights_crap, and ufc_fighters